# Practicals for lecture 1.1 - Part2

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vigji/python-cimec/blob/main/practicals/Practicals_1.1-Part2.ipynb)

## More on `numpy`

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

#### 1.1.1 Vectorizations and indexing

In [ ]:
import requests
import json

def download_meteo_data(start_date="2022-01-01", end_date="2022-12-31",
                        latitude="45.88204", longitude="11.03647",
                        data="temperature_2m"):
    """Download meteo historical data from open-meteo.com.
    
    Parameters
    ----------
        start_date : str
            Beginning of time series.
            
        end_date : str
            End of time series.
            
        latitude : float
            Latitude of the time series.
            
        longitude : float
            Longitude of the time series.
            
        data : str
            Data to download. One of "temperature_2m", "relativehumidity_2m",
            "precipitation", "snowfall", "windspeed_10m".
            
    Returns
    -------
        np.array
            1D array of timestamps
        np.array
            1D array of data, sampled every hour (24 points per day)

    """
    BASE_URL = "https://archive-api.open-meteo.com/v1/"
    query = f"archive?latitude={latitude}&longitude={longitude}&start_date={start_date}&end_date={end_date}&hourly={data}"

    r = requests.get(BASE_URL + query)
    json_dict = json.loads(r.text)
    
    if "hourly" not in json_dict.keys():
        print(json_dict)
        return None, None
    else:
        return (np.array(json_dict["hourly"][k]) for k in ["time", data])


tststamps_array, temp_array = download_meteo_data()
# Reshape the array to be a matrix of shape (n_days, n_hours). 
n_days = 365
n_hours = 24

temp_matrix = np.reshape(temp_array, (n_days, n_hours))

In [ ]:
# Let's have a look at daily excursions instead of absolute temperatures!

# Write a function that takes as input a matrix and subtracts from each row 
# the minimum value of that row, in a loop. 
# Make sure you do not change the original matrix when running the function!


In [ ]:
# Now, write a second function that does the same in a single vector operation:


In [ ]:
# Then, test it over the temperature data matrix. Use plt.matshow to visualize it before and after
# the offset subtraction:


In [ ]:
# use np.argmax over temp_matrix to find the index of the warmest hour of 2022.
# Then, unravel the index to know which day and which hour of the day it was!


In [ ]:
# We can use the index we find over one array to index a different array.

# Use argmax to find the index of the warmest hour in the (non-reshaped) temp_array.
# Then, use the index over tststamps_array to read out the corresponding timestamp.
# check that it matches your result from the previous exercise!


In [ ]:
# (Optional: we did not do this in class!)
# We can use the np.argsort() function to produce the indexes array required to
# order an array in ascending or descending values.

# For example:
random_arr = np.array([0.1, 5, 3.4, 2.3])
ordering_idxs = np.argsort(random_arr)
random_arr[ordering_idxs]  # with this index, this is now ordered!

In [ ]:
# Let's make a ranking of the 5 warmest hours of 2022! 
# Sort the (non-reshaped) temperature array using the indexes produced by np.argsort.
# so that the first elements are the highest temperatures.
# Then sort the imestamps array with the same indexes, and take the first 5.
#
# Double check you with the result that you have got in the exercises above!

